# Iteration 2 <br/>*Imputation and Cross Validation*

Jupyter Notebook referenced from my website:
[Software Nirvana: Cross Validation (2)](https://sdiehl28.netlify.com/2018/03/cross-validation-2/)

### Goals
1. Iteratively improve upon model using Age Imputation
2. Demonstrate the right and wrong way to perform Cross Validation

### Where We Are
In the first iteration, we created a simple model and showed that the accuracy was better than the null model.  The null model is the model that predicts the predominant class in all cases.

### What's Next
<a href="https://en.wikipedia.org/wiki/Imputation_(statistics)">Imputation on Wikipedia</a>

This notebook will impute the missing values for Age and use Age as an additional attribute for prediction.  We will also check to see if adding the Age variable improved prediction accuracy.

Special attention will be paid to avoid a common beginner's mistake, which is to look at the test data when performing imputation or other preprocessing steps.  The easiest way to ensure there is no "test set leakage", is to use a Pipeline.

### Common Imports and Notebook Setup

In [151]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
%matplotlib inline
sns.set() # enable seaborn style

### Previous Model Building Iteration

In [152]:
# read in all the labeled data
all_data = pd.read_csv('../data/train.csv')

# break up the dataframe into X and y
X = all_data.drop('Survived', axis=1)
y = all_data['Survived']

# As before, remove all non-numeric fields and PassengerId
drop_fields = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'PassengerId']
X = X.drop(drop_fields, axis=1)

X.dtypes

Pclass      int64
Age       float64
SibSp       int64
Parch       int64
Fare      float64
dtype: object

### Impute Age: Cross Validation the Right Way
This will be performed manually to emphasize how preprocessing operations work on the folds in a cross validation.  The key is that the held-out data, aka the test data, can never be looked at.

Later this will also be performed with an Imputer and a Pipeline to show a more concise workflow.

To a beginner, it can appear that using an Imputer and a Pipeline is a lot of extra work.  Why not just impute the Age before cross validation and be done with it.  If you were to look that at the ["Kernels"](https://www.kaggle.com/c/titanic/kernels?sortBy=votes&group=everyone&pageSize=20&competitionId=3136) on Kaggle posted for the Titantic dataset, you would see that most people do just that.  But this is bad practice and may lead to an estimate of model accuracy that is too high.  Looking at the test data prior to training your model is called "data leakage".

### Impute Age Manually (no Pipeline)
A nice introduction to overfitting, train/test split, and cross validation in Python is:[Train/Test Split and Cross Validation](https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6)

K-Fold Cross Validation *is* train/test split, but performed K times to get a more accurate estimate of model accuracy.

We train the model on the train data and we test the accuracy of the model on the test data.

With a train/test split, we have missing Age values in both the train and test sets.  

For the train set, we replace the missing values with the average Age of the train set.  

For the test set, we are not permitted to look at its data as the purpose of the test set is for model evaluation.  If we look at its data, we have made the mistake of "data leakage".  The test data has "leaked" into our model building process.  This may cause our estimate of model accuracy to be too high.  For the test set, we replace the missing values with the average Age of the *train* set. 

All of the above holds true for Cross Validation as well, as Cross Validation is just multiple train/test splits.

For each of the K folds, we will compute the mean Age value in the train set, and use that value to replace the missing values in both the train and test sets.

In [153]:
# imports
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, KFold

In [154]:
# Set random_state to get the same folds each time we call KFold()
random_state = 121212

# use low number of splits for illustration, 5 or 10 is the recommended value
k_folds = 2

# prepare for cross validation
crossvalidation = KFold(n_splits=k_folds, shuffle=True, random_state=random_state)

In [155]:
# use the K folds to impute Age and compute the accuracy score
my_scores = np.zeros(k_folds)
i = 0
lr_model = LogisticRegression()
for train_idx, test_idx in crossvalidation.split(X):
    # train subset
    X_train = X.iloc[train_idx, :].copy()
    y_train = y[train_idx].copy()
    
    # test subset
    X_test = X.iloc[test_idx, :].copy()
    y_test = y[test_idx].copy()
    
    # find the average age on the train set
    train_age_mean = X_train['Age'].mean()
    
    # use this value for *both* the train and test set
    X_train.loc[X_train['Age'].isnull(), 'Age'] = train_age_mean
    X_test.loc[X_test['Age'].isnull(), 'Age'] = train_age_mean # Key Concept!
    
    # fit model on train
    lr_model.fit(X_train, y_train)
    
    # predict using model on test
    predictions = lr_model.predict(X_test)
    
    # evaluate accuracy
    my_scores[i] = accuracy_score(y_test, predictions)
    i += 1

### Aside: Pandas copy
In the above, we had to use .copy() for our train and test sets.  This is critical to avoiding the Pandas warning: SettingWithCopyWarning.

The imputation requires us to modify both the train and test sets for X.  However X.iloc[] and X.loc[] both return a *view* into X, not a copy of a subset of X.  If we try to use this view to modify data, we will get: SettingwithCopyWarning.

In most cases this warning means your code will not do what you intended.  Therefore you should always write code that does not produce this Pandas warning.

It can be difficult to discover why this warning was issued.  One way to track this down is to print out the .is_copy member of your dataframe.  If you find you get this warning when trying to modify data through a view (implemented as a weak reference) then you need to get an independent copy of your data using .copy() and the warning will go away.

In [156]:
print(X.iloc[train_idx, :].is_copy) # view into dataframe
print(X.iloc[train_idx, :].copy().is_copy) # independent copy of dataframe

<weakref at 0x7f24ba072548; to 'DataFrame' at 0x7f24bb90c780>
None


### Impute Age with Pipeline
Scikit Learn correctly uses the mean of the train set as the replacement value for missing values in both the train and test sets.  However this is all done behind the scenes.  The following is exactly the same as the above, but requires much less code.

In [157]:
# Setup: same as above

# Set random_state to get the same folds each time we call KFold()
random_state = 121212

# use low number of splits for illustration, 5 or 10 is the recommended value
k_folds = 2

# prepare for cross validation
crossvalidation = KFold(n_splits=k_folds, shuffle=True, random_state=random_state)

In [158]:
# Use an Imputer and a Pipeline
# Note: Age is the only column in X with null values
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy='mean')

from sklearn.pipeline import make_pipeline
classifier = make_pipeline(imputer, LogisticRegression())

# cross_val_score() will properly compute 
# imputation and score per fold
scores = cross_val_score(classifier, X, y, cv=crossvalidation, 
                         scoring='accuracy', n_jobs=1)

# Check to see that we got the same scores 
# as in the above for-loop
print("Scores Match: ", (scores == my_scores).all())
print('Scores :', scores)
print(f'Cross Validated Accuracy: {scores.mean() :.3f}  SD: {scores.std() :.3f}')

Scores Match:  True
Scores : [0.70852018 0.69438202]
Cross Validated Accuracy: 0.701  SD: 0.007


### Both Methods Produce the Same Result
The for-loop over each K fold train/test split is the same as using an Imputer and LogisticRegression in a Pipeline in cross_val_score().

### Impute Age: Cross Validation The Wrong Way
In my review of Kaggle Kernels published for the Titantic dataset most of the ["Kernels"](https://www.kaggle.com/c/titanic/kernels?sortBy=votes&group=everyone&pageSize=20&competitionId=3136) imputed the Age value over the entire data set, that is, they had "data leakage".

For imputation, "data leakage" usually doesn't impact the estimate of model accuracy enough to make a practical difference.  Nevertheless, it is bad practice and if you decide to impute missing values across your entire dataset, you should explicitly note it as well as why you expect it will make little difference.

In the following I will show that the approach to imputing missing values using the entire data set produces different results than the approach which avoids "data leakage" entirely.

In [159]:
# Prior to Cross Validation:
#   impute the missing values as the mean of *all* the data
# This is "data leakage"!
# Replace all null Age values with the mean of all Age Values
X.loc[X['Age'].isnull(), 'Age'] = X['Age'].mean()

In [160]:
# Setup: same as above
random_state = 121212
n_splits = 2 # for illustration only, 5 or 10 is the recommended value
crossvalidation = KFold(n_splits=n_splits, shuffle=True, 
                        random_state=random_state)

In [161]:
# Use cross_val_score()
wrong_scores = cross_val_score(LogisticRegression(), X, y, 
                               cv=crossvalidation,
                               scoring='accuracy', n_jobs=1)

In [162]:
# We do *not* get the same scores as above
# due to data leakage
print("Scores Match: ", (scores == wrong_scores).all())
print("Scores Diff: ", scores.mean() - wrong_scores.mean())

print('Wrong Scores :', wrong_scores)
print(f'Cross Validated Accuracy: {wrong_scores.mean() :.3f}  SD: {wrong_scores.std() :.3f}')

Scores Match:  False
Scores Diff:  -2.519272434109432e-06
Wrong Scores : [0.70627803 0.69662921]
Cross Validated Accuracy: 0.701  SD: 0.005


Imputing all the missing values, over both the train and test sets, prior to performing cross validation produced different results!  That said, the difference was much less than one standard deviation, so it was not statistically significant.

The use of the Imputer and estimator in the Pipeline, or the hand coded for-loop over the K folds, produced the correct (no data leak) result.

### Cross Validation The Wrong Way: Discussion
**What was wrong:** Used all data for imputation (did not have hold-out set).

**What may happen:** Estimate of model accuracy may be too high.

**Importance in Practice:  With *imputation*, this is usually not a problem** unless you have a very small amount of data on which to perform the imputation.  In the above, we saw it made almost no difference.

**Importance in Practice:  With *feature selection*, this is usually a very serious problem** that leads to highly inflated values of model accuracy.  Feature selection is the process of determining which variables to include in the model.  If you use the entire dataset, and decide to keep variables based on some statistic of the data, such as correlation to the target variable, then you *must* use a Pipeline or otherwise ensure that you choice of variables is determined on a train set and evaluated on a test set.

**Great Explanation and Story by Robert Tibshirani:**
Robert Tibshirani, in the youtube video [Cross Validation: Right and Wrong](https://www.youtube.com/watch?v=S06JpVoNaA0&list=PL5-da3qGB5IA6E6ZNXu7dp89_uv8yocmf), explains the right and wrong way to perform cross validation in detail.  He also presents a wonderful story about a Ph.D. oral dissertation presenter filtering away variables *prior* to performing cross validation and the serious effect it had on his medical research.

### Compute the Score with K=5 for a (possibly) better accuracy estimate

In [163]:
# Compute 5x2CV Scores for iteration 2
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, RepeatedKFold
k_folds = 2
repeats = 5
random_seed = 108
crossvalidation = RepeatedKFold(n_splits=k_folds, n_repeats=repeats, 
                                random_state=random_seed)

classifier = LogisticRegression()
scores = cross_val_score(classifier, X, y, cv=crossvalidation, 
                         scoring='accuracy', n_jobs=-1)

print('Scores: ', np.round(scores, 3))
print(f'Cross Validated Accuracy: {scores.mean():.3f}  SD: {scores.std():.3f}')

Scores:  [0.7   0.719 0.693 0.706 0.697 0.717 0.709 0.71  0.693 0.706]
Cross Validated Accuracy: 0.705  SD: 0.009


In [170]:
from scipy.stats import ttest_rel
first_iter_scores = np.load('../data/iter01.data.npy')

ttest_rel(first_iter_scores, scores)

Ttest_relResult(statistic=-5.506462620863888, pvalue=0.0003770210948230576)

In [286]:
import scipy.stats as ss
y = scores - first_iter_scores
print(y.mean(), y.std())

0.019756134428377092 0.010763426062416977


In [288]:
ss.wilcoxon(first_iter_scores, scores)

WilcoxonResult(statistic=0.0, pvalue=0.005062032126267864)

In [280]:
rvs1 = stats.norm.rvs(loc=5,scale=10,size=50)
rvs2 = (stats.norm.rvs(loc=5,scale=10,size=50) +
        stats.norm.rvs(loc=5,scale=0.0000001,size=50))
stats.ttest_rel(rvs1,rvs2)

Ttest_relResult(statistic=-3.9559606189130236, pvalue=0.0002454940254389779)

### Previous Model Building Iteration

In [27]:
# read in all the labeled data
all_data = pd.read_csv('../data/train.csv')

# break up the dataframe into X and y
X = all_data.drop('Survived', axis=1)
y = all_data['Survived']

# As before, remove all non-numeric fields and PassengerId
drop_fields = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'PassengerId']
X = X.drop(drop_fields, axis=1)

X.dtypes

Pclass      int64
Age       float64
SibSp       int64
Parch       int64
Fare      float64
dtype: object

In [40]:
random_state = 100
n_splits = 5 # 5 or 10 is the recommended value
crossvalidation = KFold(n_splits=n_splits, shuffle=True, 
                        random_state=random_state)

imputer = Imputer(strategy='mean')
classifier = make_pipeline(imputer, LogisticRegression())
scores = cross_val_score(classifier, X, y, cv=crossvalidation, 
                         scoring='accuracy', n_jobs=1)

# Use the mean score as the best estimate of model accuracy
print('Scores: ', scores)
print(f'Cross Validated Accuracy: {scores.mean() :.3f}  SD: {scores.std() :.3f}')

Scores:  [0.7150838  0.69662921 0.70224719 0.74719101 0.64606742]
Cross Validated Accuracy: 0.701  SD: 0.033


K=2 and K=5 gave use almost the same estimate of model accuracy in this case.

### Model Summary
Model building steps only.

In [149]:
# read in all the labeled data
all_data = pd.read_csv('../data/train.csv')

# break up the dataframe into X and y
X = all_data.drop('Survived', axis=1)
y = all_data['Survived']

# As before, remove all non-numeric fields and PassengerId
drop_fields = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'PassengerId']
X = X.drop(drop_fields, axis=1)

random_state = 100
k_folds = 5 # 5 or 10 is the recommended value
crossvalidation = KFold(n_splits=k_folds, shuffle=True, 
                        random_state=random_state)

imputer = Imputer(strategy='mean')
classifier = make_pipeline(imputer, LogisticRegression())
scores = cross_val_score(classifier, X, y, cv=crossvalidation, 
                         scoring='accuracy', n_jobs=1)

# Use the mean score as the best estimate of model accuracy
print('Scores: ', scores)
print(f'Cross Validated Accuracy: {scores.mean() :.3f}  SD: {scores.std() :.3f}')

# How does this compare to previous iteration?
# previous iteration: .685 SD: .031
# this iteration:     .701 SD: .033
# The scores differ by less than 1 SD, so this is not statistically significant
# As expected, the p-value for the independent 2-sample t-test is greater than .05
prev_iter_scores = [0.72625698, 0.70786517, 0.63483146, 0.6741573, 0.67977528]
from scipy import stats
print(stats.ttest_ind(scores, prev_iter_scores))

Scores:  [0.7150838  0.69662921 0.70224719 0.74719101 0.64606742]
Cross Validated Accuracy: 0.701  SD: 0.033
Ttest_indResult(statistic=0.7448540119494089, pvalue=0.4776685207154546)


### Summary

In this iteration we:
* showed the right and wrong way to perform cross validation
* showed that using Pipelines with Cross Validation improves the quality of the software by making it easy to concisely perform cross validation correctly.
* added the Age variable to our model and used it's mean to impute missing values
* showed that the new estimate of model accuracy is not significantly different than the previous iteration